In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
os.getcwd()

'D:\\GitHubProjects\\covid_mortality_prediction'

In [5]:
data_dir = '..\\synthea_covid19_data\\100k_synthea_covid19_csv\\'
data_dir

'..\\synthea_covid19_data\\100k_synthea_covid19_csv\\'

# Explore 'patients' table

In [6]:
patients = pd.read_csv(data_dir+'patients.csv', nrows=20)

In [7]:
patients.columns

Index(['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')

In [8]:
patients.sample()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
14,29581d13-2687-402f-9c4c-06141a319e61,2000-12-19,NaN,999-28-2716,S99985509,NaN,Ms.,Carmelia328,Konopelski743,NaN,...,Lee Massachusetts US,1025 Collier Arcade,Ashland,Massachusetts,Middlesex County,NaN,42.291986,-71.463724,439502.18,72357.13


In [9]:
# Remove a few unnecessary columns to allow for easier visualization
patients.drop(['SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN'], axis=1, inplace=True)

In [10]:
patients.sample()

,Id,BIRTHDATE,DEATHDATE,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
17,e52bd28f-82df-41da-97f1-556ecb8514d2,1958-07-29,NaN,S,white,nonhispanic,F,Watertown Massachusetts US,970 Grant Highlands,Foxborough,Massachusetts,Norfolk County,NaN,42.078942,-71.225442,1400772.46,9877.44


For now, will only keep basic demographic information and remove the rest. 
HEALTHCARE_COVERAGE may be a foreign key so keeping it for now 
HEALTHCARE_EXPENSES will be removed to prevent data leakage

In [11]:
# 
patients.drop(['BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE','COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES'], axis=1, inplace=True)

In [12]:
patients.sample()

,Id,BIRTHDATE,DEATHDATE,MARITAL,RACE,ETHNICITY,GENDER,HEALTHCARE_COVERAGE
6,9d873139-72ec-4711-9ebd-f278e100a499,1989-06-22,NaN,S,white,hispanic,M,370.8


Now read in the entire patient.csv file with only the relevant columns

In [13]:
patients = pd.read_csv(data_dir+'patients.csv', usecols=['Id', 'BIRTHDATE', 'DEATHDATE', 'MARITAL', 'RACE', 'ETHNICITY',
                                                         'GENDER', 'HEALTHCARE_COVERAGE'])

In [14]:
patients.sample()

,Id,BIRTHDATE,DEATHDATE,MARITAL,RACE,ETHNICITY,GENDER,HEALTHCARE_COVERAGE
48540,58f0d200-a3a4-4bfa-b3dd-7a9723a3179f,1932-11-09,1985-05-30,M,black,nonhispanic,M,4449.6


In [15]:
def feature_summary(dataset):
    feature_summary = pd.DataFrame()
    feature_summary['Data Type'] = dataset.dtypes
    feature_summary['Num Unique'] = dataset.nunique()
    feature_summary['Num Missing'] = dataset.isnull().sum()
    feature_summary['% Missing'] = round((feature_summary['Num Missing'] / len(dataset.index)) * 100, 2)

    # Grab info from dataset.describe()
    numerical_var_info = dataset.describe().T
    feature_summary['Min'] = np.round(numerical_var_info['min'], 2)
    feature_summary['Max'] = np.round(numerical_var_info['max'], 2)
    feature_summary['Mean'] = np.round(numerical_var_info['mean'], 2)
    feature_summary['Median'] = np.round(numerical_var_info['50%'], 2)
    feature_summary['Std'] = np.round(numerical_var_info['std'], 2)
    
    return feature_summary

In [16]:
feature_summary(patients)

,Data Type,Num Unique,Num Missing,% Missing,Min,Max,Mean,Median,Std
Id,object,124150,0,0.00,NaN,NaN,NaN,NaN,NaN
BIRTHDATE,object,31098,0,0.00,NaN,NaN,NaN,NaN,NaN
DEATHDATE,object,12318,100000,80.55,NaN,NaN,NaN,NaN,NaN
MARITAL,object,2,36121,29.09,NaN,NaN,NaN,NaN,NaN
RACE,object,5,0,0.00,NaN,NaN,NaN,NaN,NaN
ETHNICITY,object,2,0,0.00,NaN,NaN,NaN,NaN,NaN
GENDER,object,2,0,0.00,NaN,NaN,NaN,NaN,NaN
HEALTHCARE_COVERAGE,float64,74160,0,0.00,0.0,2146427.06,13004.98,6003.64,42643.27


In [17]:
patients.head()

,Id,BIRTHDATE,DEATHDATE,MARITAL,RACE,ETHNICITY,GENDER,HEALTHCARE_COVERAGE
0,1ff7f10f-a204-4bb1-aa72-dd763fa99482,2017-08-24,NaN,NaN,white,nonhispanic,M,1499.08
1,9bcf6ed5-d808-44af-98a0-7d78a29ede72,2016-08-01,NaN,NaN,white,nonhispanic,F,1870.72
2,5163c501-353c-4a82-b863-a3f1df2d6cf1,2004-01-09,NaN,NaN,white,nonhispanic,F,3131.44
3,cc3c806f-4a09-4a89-a990-4286450956be,1996-11-15,NaN,NaN,white,nonhispanic,M,3632.96
4,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,2019-06-12,NaN,NaN,white,nonhispanic,F,903.28


In [18]:
# Clear memory, explore next table
del patients

# Explore 'encounters' table

In [19]:
encounters = pd.read_csv(data_dir+'encounters.csv', nrows=20)

In [20]:
encounters.columns

Index(['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER',
       'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')

In [21]:
encounters.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,d317dacb-d801-4136-be7d-440094f7ae83,2019-02-16T01:43:20Z,2019-02-16T01:58:20Z,1ff7f10f-a204-4bb1-aa72-dd763fa99482,5103c940-0c08-392f-95cd-446e0cea042a,8b532fbe-4254-3a60-a442-33028916d24e,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,185345009,Encounter for symptom,129.16,129.16,69.16,65363002.0,Otitis media
1,52051c30-c6c3-45fe-b5da-a790f1680e91,2019-08-02T01:43:20Z,2019-08-02T02:13:20Z,1ff7f10f-a204-4bb1-aa72-dd763fa99482,0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3,872f1318-45f7-34cc-adc7-26d59fc7c203,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
2,759cfd9c-b40b-45df-a5b2-fca418027a6a,2019-10-31T01:43:20Z,2019-10-31T01:58:20Z,1ff7f10f-a204-4bb1-aa72-dd763fa99482,5103c940-0c08-392f-95cd-446e0cea042a,8b532fbe-4254-3a60-a442-33028916d24e,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,185345009,Encounter for symptom,129.16,129.16,69.16,65363002.0,Otitis media
3,20032279-0deb-41f9-bea4-93710396eb95,2020-01-31T01:43:20Z,2020-01-31T01:58:20Z,1ff7f10f-a204-4bb1-aa72-dd763fa99482,0b9f3f7c-8ab6-30a5-b3ae-4dc0e0c00cb3,872f1318-45f7-34cc-adc7-26d59fc7c203,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
4,60584c6a-e26b-4176-a660-86ad84c7ceca,2020-03-02T01:43:20Z,2020-03-02T02:39:20Z,1ff7f10f-a204-4bb1-aa72-dd763fa99482,fd328395-ab1d-35c6-a2d0-d05a9a79cf11,595c4f10-bcce-3892-a5d2-f95e3ab2ab74,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,ambulatory,185345009,Encounter for symptom (procedure),129.16,129.16,69.16,NaN,NaN


Not sure if I want to use organization, provider, or payer. I'll take a look at those tables first

In [22]:
organizations = pd.read_csv(data_dir+'organizations.csv', nrows=20)
providers = pd.read_csv(data_dir+'providers.csv', nrows=20)
payers = pd.read_csv(data_dir+'payers.csv', nrows=20)

In [23]:
organizations.head()

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
0,ef58ea08-d883-3957-8300-150554edc8fb,HEALTHALLIANCE HOSPITALS INC,60 HOSPITAL ROAD,LEOMINSTER,MA,1453,42.520838,-71.770876,9784662000,1.824774e+07,145353
1,69176529-fd1f-3b3f-abce-a0a3626769eb,MOUNT AUBURN HOSPITAL,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,2138,42.375967,-71.118275,6174923500,3.485678e+07,277220
2,5e765f2b-e908-3888-9fc7-df2cb87beb58,STURDY MEMORIAL HOSPITAL,211 PARK STREET,ATTLEBORO,MA,2703,41.931653,-71.294503,5082225200,2.552913e+07,203115
3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,LAWRENCE GENERAL HOSPITAL,ONE GENERAL STREET,LAWRENCE,MA,1842,42.700273,-71.161357,9786834000,3.142294e+07,249999
4,e002090d-4e92-300e-b41e-7d1f21dee4c6,CAMBRIDGE HEALTH ALLIANCE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,2138,42.375967,-71.118275,6176652300,3.507684e+07,279068


In [24]:
providers.head()

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE,ZIP,LAT,LON,UTILIZATION
0,4781b3e5-e13c-3ef0-945d-649abe044d47,ef58ea08-d883-3957-8300-150554edc8fb,Jeffrey461 Beer512,M,GENERAL PRACTICE,60 HOSPITAL ROAD,LEOMINSTER,MA,1453,42.520838,-71.770876,145353
1,a50f1afd-2a19-3f0f-a3f0-d9d8e51c6f9b,69176529-fd1f-3b3f-abce-a0a3626769eb,Allison818 Bednar518,F,GENERAL PRACTICE,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,2138,42.375967,-71.118275,277220
2,692c8e0d-d680-3c70-9dfd-dfc2cacf3db5,5e765f2b-e908-3888-9fc7-df2cb87beb58,Angelena945 Torp761,F,GENERAL PRACTICE,211 PARK STREET,ATTLEBORO,MA,2703,41.931653,-71.294503,203115
3,f4f87207-b74e-390e-b1c9-ea000db107c3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,Denver542 Trantow673,M,GENERAL PRACTICE,ONE GENERAL STREET,LAWRENCE,MA,1842,42.700273,-71.161357,249999
4,75322072-9b0a-302a-b343-326fd18aeb1c,e002090d-4e92-300e-b41e-7d1f21dee4c6,Carlos172 Rath779,M,GENERAL PRACTICE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,2138,42.375967,-71.118275,279068


In [25]:
payers.head()

,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE,AMOUNT_COVERED,AMOUNT_UNCOVERED,REVENUE,...,UNCOVERED_ENCOUNTERS,COVERED_MEDICATIONS,UNCOVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_PROCEDURES,COVERED_IMMUNIZATIONS,UNCOVERED_IMMUNIZATIONS,UNIQUE_CUSTOMERS,QOLS_AVG,MEMBER_MONTHS
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323,4.531388e+07,2.485856e+07,1.953770e+08,...,0,150159,0,82736,0,34249,0,3071,0.490522,484500
1,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,7500 Security Blvd,Baltimore,MD,21244.0,1-800-633-4227,5.846347e+08,1.018902e+08,9.435702e+08,...,0,1455673,0,911862,0,337691,0,25012,0.785927,3931632
2,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323,3.315169e+08,1.821008e+08,1.994596e+09,...,0,607074,0,5071067,0,665206,0,52994,0.770179,8548752
3,d47b3510-2895-3b70-9897-342d681c769d,Humana,500 West Main St,Louisville,KY,40018.0,1-844-330-7799,8.520899e+07,1.056451e+08,1.479080e+10,...,222875,0,591617,1560577,75944,778250,22797,32041,0.920409,8769672
4,6e2f1a2d-27bd-3701-8d08-dae202c58632,Blue Cross Blue Shield,Michigan Plaza,Chicago,IL,60007.0,1-800-262-2583,2.188510e+08,1.419202e+08,1.502221e+10,...,0,586516,0,1560056,0,769640,0,31127,0.918575,8414820


In [26]:
conditions = pd.read_csv(data_dir+'conditions.csv', nrows=20)

In [27]:
conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2019-02-15,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,d317dacb-d801-4136-be7d-440094f7ae83,65363002,Otitis media
1,2019-10-30,2020-01-30,1ff7f10f-a204-4bb1-aa72-dd763fa99482,759cfd9c-b40b-45df-a5b2-fca418027a6a,65363002,Otitis media
2,2020-03-01,2020-03-30,1ff7f10f-a204-4bb1-aa72-dd763fa99482,60584c6a-e26b-4176-a660-86ad84c7ceca,386661006,Fever (finding)
3,2020-03-01,2020-03-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,60584c6a-e26b-4176-a660-86ad84c7ceca,840544004,Suspected COVID-19
4,2020-03-01,2020-03-30,1ff7f10f-a204-4bb1-aa72-dd763fa99482,60584c6a-e26b-4176-a660-86ad84c7ceca,840539006,COVID-19


In [28]:
observations = pd.read_csv(data_dir+'observations.csv', nrows=20)

In [29]:
observations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,8302-2,Body Height,82.7,cm,numeric
1,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,29463-7,Body Weight,11.5,kg,numeric
3,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,77606-2,Weight-for-length Per age and sex,47.0,%,numeric
4,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,9843-4,Head Occipital-frontal circumference,46.9,cm,numeric


In [30]:
allergies = pd.read_csv(data_dir+'allergies.csv', nrows=20)
careplans = pd.read_csv(data_dir+'careplans.csv', nrows=20)
devices = pd.read_csv(data_dir+'devices.csv', nrows=20)
imaging_studies = pd.read_csv(data_dir+'imaging_studies.csv', nrows=20)
immunizations = pd.read_csv(data_dir+'immunizations.csv', nrows=20)
medications = pd.read_csv(data_dir+'medications.csv', nrows=20)
payer_transitions = pd.read_csv(data_dir+'payer_transitions.csv', nrows=20)
procedures = pd.read_csv(data_dir+'procedures.csv', nrows=20)
supplies = pd.read_csv(data_dir+'supplies.csv', nrows=20)

In [31]:
allergies.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1983-01-22,NaN,da454f92-9e90-43f8-9514-550fbaa6de3d,ddebd8f1-c2fa-4958-a19e-2e55810bcfd6,424213003,Allergy to bee venom
1,1983-01-22,NaN,da454f92-9e90-43f8-9514-550fbaa6de3d,ddebd8f1-c2fa-4958-a19e-2e55810bcfd6,418689008,Allergy to grass pollen
2,1983-01-22,NaN,da454f92-9e90-43f8-9514-550fbaa6de3d,ddebd8f1-c2fa-4958-a19e-2e55810bcfd6,419263009,Allergy to tree pollen
3,1983-01-22,NaN,da454f92-9e90-43f8-9514-550fbaa6de3d,ddebd8f1-c2fa-4958-a19e-2e55810bcfd6,417532002,Allergy to fish
4,2002-02-24,NaN,29581d13-2687-402f-9c4c-06141a319e61,1d1e8b9d-00c5-47dc-a635-16769715d7cc,91934008,Allergy to nut


In [32]:
careplans.head()

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,ed87cb74-1c58-482c-a809-36307a961955,2020-03-01,2020-03-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,60584c6a-e26b-4176-a660-86ad84c7ceca,736376001,Infectious disease care plan (record artifact),840544004,Suspected COVID-19
1,3d0f0d4a-03f7-4714-a9fd-04634af395f0,2020-03-01,2020-03-30,1ff7f10f-a204-4bb1-aa72-dd763fa99482,60584c6a-e26b-4176-a660-86ad84c7ceca,736376001,Infectious disease care plan (record artifact),840539006,COVID-19
2,666f9af9-e8ed-4475-bc0a-bbb255b1c773,2020-02-12,2020-02-26,9bcf6ed5-d808-44af-98a0-7d78a29ede72,5d9dfe80-7edf-4181-9753-770e361934d1,91251008,Physical therapy procedure,44465007,Sprain of ankle
3,21f3ecc9-32d0-4926-a939-e076a1d813a4,2020-03-13,2020-03-13,9bcf6ed5-d808-44af-98a0-7d78a29ede72,a78e78d9-33bb-40bc-9e42-e47ff7e910cc,736376001,Infectious disease care plan (record artifact),840544004,Suspected COVID-19
4,7084b92a-9c76-4f3e-ac1d-3b55da945699,2020-03-13,2020-04-14,9bcf6ed5-d808-44af-98a0-7d78a29ede72,a78e78d9-33bb-40bc-9e42-e47ff7e910cc,736376001,Infectious disease care plan (record artifact),840539006,COVID-19


In [33]:
devices.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,UDI
0,1986-07-04,NaN,7e2c6949-dce9-44be-b6ef-50aca95840d7,88f42ade-3921-4432-b53c-20f2231a6e87,72506001,Implantable defibrillator device (physical ob...,(01)32509593087287(11)860613(17)110628(10)6374...
1,2020-03-05,2020-03-11,a89e2d41-db41-4bf2-8e7d-f315743219dd,47d7e44e-94df-461f-b8a3-d84f07541fe1,449071006,Mechanical ventilator (physical object),(01)51467824701974(11)200213(17)450227(10)6798...
2,2020-03-05,2020-03-05,a89e2d41-db41-4bf2-8e7d-f315743219dd,47d7e44e-94df-461f-b8a3-d84f07541fe1,448907002,Videolaryngoscope (physical object),(01)46214226835296(11)200213(17)450227(10)6151...
3,2020-03-05,2020-03-05,a89e2d41-db41-4bf2-8e7d-f315743219dd,47d7e44e-94df-461f-b8a3-d84f07541fe1,36965003,Hemodialysis machine device (physical object),(01)26706893695172(11)200213(17)450227(10)8603...
4,2020-03-06,2020-03-06,a89e2d41-db41-4bf2-8e7d-f315743219dd,47d7e44e-94df-461f-b8a3-d84f07541fe1,36965003,Hemodialysis machine device (physical object),(01)14121876319892(11)200214(17)450228(10)8066...


In [34]:
imaging_studies.head()

,Id,DATE,PATIENT,ENCOUNTER,BODYSITE_CODE,BODYSITE_DESCRIPTION,MODALITY_CODE,MODALITY_DESCRIPTION,SOP_CODE,SOP_DESCRIPTION
0,fe380e56-a583-4133-90e2-f2519b973996,1983-08-10,7e2c6949-dce9-44be-b6ef-50aca95840d7,e20e918a-c65a-4f28-9041-9dc331a30373,344001,Ankle,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage
1,bf567838-d61d-4862-9921-fbb79b494cb2,2008-10-03,8742348d-72b0-40b3-a970-08e50b30e256,73aa0361-5499-469f-b80d-afeb0ccbe6db,261179002,thoracic,US,Ultrasound,1.2.840.10008.5.1.4.1.1.3.1,Ultrasound Multiframe Image Storage
2,c210ff52-e285-43e5-8612-d202914f9913,2009-04-01,8742348d-72b0-40b3-a970-08e50b30e256,986a5bae-3e06-4fae-a7dc-d6d30b4d4a65,51185008,Thoracic structure (body structure),CR,Computed Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage – for Presentation
3,3822e2be-5ff2-40a0-9e79-a241bbfee498,2009-04-01,8742348d-72b0-40b3-a970-08e50b30e256,f6a103ad-0a86-4224-b2d3-afb3095e0444,261179002,Thoracic,US,Ultrasound,1.2.840.10008.5.1.4.1.1.3.1,Ultrasound Multiframe Image Storage
4,cad5e6e6-a50b-4bca-8073-99ea37816066,2009-04-01,8742348d-72b0-40b3-a970-08e50b30e256,f6a103ad-0a86-4224-b2d3-afb3095e0444,51185008,Thoracic structure,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage – for Presentation


In [35]:
immunizations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST
0,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,140,Influenza seasonal injectable preservative ...,140.52
1,2020-01-30,1ff7f10f-a204-4bb1-aa72-dd763fa99482,20032279-0deb-41f9-bea4-93710396eb95,83,Hep A ped/adol 2 dose,140.52
2,2019-07-08,9bcf6ed5-d808-44af-98a0-7d78a29ede72,26f579ef-1dc8-4899-b8fc-13f98eaeee55,140,Influenza seasonal injectable preservative ...,140.52
3,2020-02-21,5163c501-353c-4a82-b863-a3f1df2d6cf1,929c6964-3c16-4813-875a-88619372e9ce,140,Influenza seasonal injectable preservative ...,140.52
4,2020-02-21,5163c501-353c-4a82-b863-a3f1df2d6cf1,929c6964-3c16-4813-875a-88619372e9ce,114,meningococcal MCV4P,140.52


In [36]:
medications.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2019-10-30,2019-11-13,1ff7f10f-a204-4bb1-aa72-dd763fa99482,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,759cfd9c-b40b-45df-a5b2-fca418027a6a,308182,Amoxicillin 250 MG Oral Capsule,7.05,0.0,1,7.05,NaN,NaN
1,2019-10-30,2019-11-13,1ff7f10f-a204-4bb1-aa72-dd763fa99482,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,759cfd9c-b40b-45df-a5b2-fca418027a6a,313820,Acetaminophen 160 MG Chewable Tablet,5.72,0.0,1,5.72,NaN,NaN
2,2020-02-12,2020-02-26,9bcf6ed5-d808-44af-98a0-7d78a29ede72,4d71f845-a6a9-3c39-b242-14d25ef86a8d,5d9dfe80-7edf-4181-9753-770e361934d1,313820,Acetaminophen 160 MG Chewable Tablet,5.25,0.0,1,5.25,NaN,NaN
3,2020-04-28,2020-05-08,9bcf6ed5-d808-44af-98a0-7d78a29ede72,4d71f845-a6a9-3c39-b242-14d25ef86a8d,794aac2b-0eb8-4c36-98bb-a4317b344e6e,834061,Penicillin V Potassium 250 MG Oral Tablet,13.90,0.0,1,13.90,43878008.0,Streptococcal sore throat (disorder)
4,2019-12-26,NaN,5163c501-353c-4a82-b863-a3f1df2d6cf1,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,66829d53-9d16-4f2c-82ad-bfb2409fdb25,978950,Natazia 28 Day Pack,29.52,0.0,5,147.60,NaN,NaN


In [37]:
payer_transitions.head()

,PATIENT,START_YEAR,END_YEAR,PAYER,OWNERSHIP
0,1ff7f10f-a204-4bb1-aa72-dd763fa99482,2017,2017,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Self
1,cc3c806f-4a09-4a89-a990-4286450956be,1996,2014,d47b3510-2895-3b70-9897-342d681c769d,Guardian
2,4dc153b6-bd07-49f3-bb4c-bad875b388a3,1992,2010,6e2f1a2d-27bd-3701-8d08-dae202c58632,Guardian
3,9d873139-72ec-4711-9ebd-f278e100a499,1989,1989,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,NaN
4,682077f3-bd6b-462d-ac1c-c394ca8240d2,1983,2001,4d71f845-a6a9-3c39-b242-14d25ef86a8d,Guardian


In [38]:
procedures.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2019-08-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,52051c30-c6c3-45fe-b5da-a790f1680e91,430193006,Medication Reconciliation (procedure),495.80,NaN,NaN
1,2020-03-01,1ff7f10f-a204-4bb1-aa72-dd763fa99482,60584c6a-e26b-4176-a660-86ad84c7ceca,261352009,Face mask (physical object),516.65,840544004.0,Suspected COVID-19
2,2020-01-06,9bcf6ed5-d808-44af-98a0-7d78a29ede72,723060b3-7be8-4d12-8e1e-04f9da2c52d5,430193006,Medication Reconciliation (procedure),453.37,NaN,NaN
3,2020-03-13,9bcf6ed5-d808-44af-98a0-7d78a29ede72,a78e78d9-33bb-40bc-9e42-e47ff7e910cc,261352009,Face mask (physical object),516.65,840544004.0,Suspected COVID-19
4,2020-04-28,9bcf6ed5-d808-44af-98a0-7d78a29ede72,794aac2b-0eb8-4c36-98bb-a4317b344e6e,117015009,Throat culture (procedure),2169.16,43878008.0,Streptococcal sore throat (disorder)


In [39]:
supplies.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,QUANTITY
0,2020-02-19,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,b7455838-3607-47f4-aaa5-fd89abea7d29,409534002,Disposable air-purifying respirator (physical ...,2
1,2020-02-19,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,b7455838-3607-47f4-aaa5-fd89abea7d29,713779008,Nitrile examination/treatment glove non-powde...,24
2,2020-02-19,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,b7455838-3607-47f4-aaa5-fd89abea7d29,469673003,Isolation gown single-use (physical object),12
3,2020-02-19,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,b7455838-3607-47f4-aaa5-fd89abea7d29,706724001,Face shield (physical object),2
4,2020-02-19,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,b7455838-3607-47f4-aaa5-fd89abea7d29,419343004,Alcohol disinfectant (substance),1
